In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import csv

In [35]:
start_year = 1980
end_year = 1982

urls = list()

for year in range(start_year, end_year + 1):
    url = f"https://www.imdb.com/search/title/?title_type=feature&release_date={year}-01-01,{year}-12-31&count=50&sort=boxoffice_gross_us,desc"
    urls.append(url)

In [36]:
print(*urls, sep='\n')

https://www.imdb.com/search/title/?title_type=feature&release_date=1980-01-01,1980-12-31&count=50&sort=boxoffice_gross_us,desc
https://www.imdb.com/search/title/?title_type=feature&release_date=1981-01-01,1981-12-31&count=50&sort=boxoffice_gross_us,desc
https://www.imdb.com/search/title/?title_type=feature&release_date=1982-01-01,1982-12-31&count=50&sort=boxoffice_gross_us,desc


In [41]:
with open("test.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f, delimiter=';')
    csv_header = ['year', 'place', 'title', 'rating', 'min',
                    'director', 'genre', 'release_date', 'origin_country',
                    'company', 'budget', 'gross_us', 'gross_world', 'open_week']
    
    writer.writerow(csv_header)

In [42]:
def get_info_from_url(html, headers, data):
    
    year = get_year(url)
    
    movie_url_start = "https://www.imdb.com"
    # response = requests.get(url, headers=headers)
    # print(response)
    soup = BeautifulSoup(html, "html.parser")
    movies = soup.select(".ipc-metadata-list-summary-item")

    for movie in movies:
        movie_data = list()
        movie_data.append(year)
        link = movie.select(".ipc-title-link-wrapper")[0]
        place = link.text.split('.')[0].strip()
        movie_data.append(place)

        movie_url = movie_url_start + link.get('href')   
        get_info_from_movie_url(movie_url, headers, movie_data)
        data.append(movie_data)
        print(movie_data[1], movie_data[2])


def get_info_from_movie_url(url, headers, movie_data):

    response = requests.get(url, headers=headers)
    # print(response)
    soup = BeautifulSoup(response.text, "html.parser")
    title = get_text_from_class(soup, ".hero__primary-text")
    # director = get_text_from_class(soup, "sc-af040695-3 dUiZpV")
   
    rating_obj = soup.find(attrs={"data-testid": "hero-rating-bar__aggregate-rating__score"})
    rating = rating_obj.find("span").text
    
    genre_obj = soup.find(class_="ipc-chip-list__scroller")
    genre_list = list()
    for genre in genre_obj.find_all(class_="ipc-chip__text"):
        genre_list.append(genre.text)
    
    director = soup.find(class_="ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link").text

    details_section = soup.find(attrs={"data-testid": "title-details-section"})
    
    release_date = details_section.find(attrs={"data-testid": "title-details-releasedate"}).find(class_="ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link")
    OriginCountry = details_section.find(attrs={"data-testid": "title-details-origin"}).find(class_="ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link")
    Company = details_section.find(attrs={"data-testid": "title-details-companies"}).find(class_="ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link")
    
    BoxOffice_section = soup.find(attrs={"data-testid": "title-boxoffice-section"})

    budget_obj = BoxOffice_section.find(attrs={"data-testid": "title-boxoffice-budget"})
    if type(budget_obj) == type(BoxOffice_section):
        budget = budget_obj.find(class_="ipc-metadata-list-item__list-content-item ipc-btn--not-interactable").text
    else:
        budget = ""

    grossdomestic_obj = BoxOffice_section.find(attrs={"data-testid": "title-boxoffice-grossdomestic"})
    if type(grossdomestic_obj) == type(BoxOffice_section):
        grossdomestic = grossdomestic_obj.find(class_="ipc-metadata-list-item__list-content-item ipc-btn--not-interactable").text
    else:
        grossdomestic = ""

    openingweekenddomestic_obj = BoxOffice_section.find(attrs={"data-testid": "title-boxoffice-openingweekenddomestic"})
    if type(openingweekenddomestic_obj) == type(BoxOffice_section):
        openingweekenddomestic = openingweekenddomestic_obj.find(class_="ipc-metadata-list-item__list-content-item ipc-btn--not-interactable").text
    else:
        openingweekenddomestic = ""

    cumulativeworldwidegross_obj = BoxOffice_section.find(attrs={"data-testid": "title-boxoffice-cumulativeworldwidegross"})
    if type(cumulativeworldwidegross_obj) == type(BoxOffice_section):
        cumulativeworldwidegross = cumulativeworldwidegross_obj.find(class_="ipc-metadata-list-item__list-content-item ipc-btn--not-interactable").text
    else:
        cumulativeworldwidegross = ""
    
    runtime_section = soup.find(attrs={"data-testid": "title-techspec_runtime"})    
    runtime_min = runtime_section.find(class_="ipc-metadata-list-item__list-content-item--subText")

    movie_data.append(title.strip())
    movie_data.append(rating.strip())
    movie_data.append(runtime_min.text.strip())
    movie_data.append(director.strip())
    movie_data.append(genre_list)
    movie_data.append(release_date.text.strip())
    movie_data.append(OriginCountry.text.strip())
    movie_data.append(Company.text.strip())
    movie_data.append(budget.strip())
    movie_data.append(grossdomestic.strip())
    movie_data.append(cumulativeworldwidegross.strip())
    movie_data.append(openingweekenddomestic.strip())

    
    
def get_text_from_class(soup, soup_class):
    a_row = soup.select(soup_class)
    if len(a_row) > 0:
        a_row = a_row[0]
        text = a_row.text
    else:
        text = ""
    return text

def get_year(url):
    key = 'release_date='
    start = url.find(key)

    if start != -1:
        year = url[start + len(key): start + len(key) + 4]
    else:
        year = '0000'
    
    return year




In [ ]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/117.0",
    "Accept-Language": "en-US,en;q=0.9",
    "Referer": "https://www.imdb.com/",
}

for url in urls:
    data = list()
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get(url)
    # time.sleep(2)
    html = driver.page_source
    driver.quit()
    get_info_from_url(html, headers, data)
    print('data', data)

    with open("test.csv", "a", newline="", encoding="utf-8") as f:
        writer = csv.writer(f, delimiter=';')
        writer.writerows(data)


In [44]:
df = pd.read_csv('test.csv', delimiter = ';')
df

,year,place,title,rating,min,director,genre,release_date,origin_country,company,budget,gross_us,gross_world,open_week
0,1980,1,Star Wars: Episode V - The Empire Strikes Back,8.7,(124 min),Irvin Kershner,"['Action Epic', 'Adventure Epic', 'Dark Fantas...","June 18, 1980 (United States)",United States,Lucasfilm,"$18,000,000 (estimated)","$292,753,960","$550,016,086","$4,910,483"
1,1980,2,Superman II,6.8,(127 min),Richard Lester,"['Alien Invasion', 'Globetrotting Adventure', ...","June 19, 1981 (United States)",United Kingdom,International Film Production,"$54,000,000 (estimated)","$108,185,706","$216,385,706","$14,100,523"
2,1980,3,9 to 5,6.9,(109 min),Colin Higgins,"['Farce', 'Comedy']","December 19, 1980 (United States)",United States,IPC Films,"$10,000,000 (estimated)","$103,290,500","$103,303,473","$3,966,832"
3,1980,4,Stir Crazy,6.7,(111 min),Sidney Poitier,"['Buddy Comedy', 'Dark Comedy', 'Comedy', 'Cri...","December 12, 1980 (United States)",United States,Columbia Pictures,"$10,000,000 (estimated)","$101,300,000","$101,300,000","$8,691,886"
4,1980,5,Airplane!,7.7,(88 min),Jim Abrahams,"['Dark Comedy', 'Farce', 'Parody', 'Satire', '...","July 2, 1980 (United States)",United States,Paramount Pictures,"$3,500,000 (estimated)","$83,453,539","$83,455,874",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,1982,46,Halloween III: Season of the Witch,5.2,(98 min),Tommy Lee Wallace,"['Artificial Intelligence', 'Body Horror', 'Cy...","October 22, 1982 (United States)",United States,Dino De Laurentiis Company,"$2,500,000 (estimated)","$14,400,000","$14,402,425","$6,333,259"
146,1982,47,Diner,7.0,(110 min),Barry Levinson,"['Period Drama', 'Comedy', 'Drama']","May 21, 1982 (United States)",United States,Metro-Goldwyn-Mayer (MGM),"$5,000,000 (estimated)","$14,099,953","$14,099,953",NaN
147,1982,48,The Beastmaster,6.2,(118 min),Don Coscarelli,"['Sword & Sorcery', 'Action', 'Adventure', 'Fa...","August 20, 1982 (United States)",West Germany,Leisure Investment Company,"$8,000,000 (estimated)","$14,056,528","$14,056,528","$2,955,641"
148,1982,49,Missing,7.7,(122 min),Costa-Gavras,"['Political Thriller', 'Tragedy', 'Biography',...","March 12, 1982 (United States)",United States,Polygram Filmed Entertainment,"$9,500,000 (estimated)","$14,000,000","$14,000,000",NaN


SyntaxError: invalid syntax (1548159263.py, line 1)